In [51]:
class Transition:
    """The Basic Transition implementation of a state is defined"""

    def __init__(self, InState, InString, OutState, OutString = None):
        """ Defines the basic transition properties and intialized with Argument
            Instate    -   Current state ID
            Instring   -   Transition string input
            OutString  -   Transition output sting
            OutState   -   Next state's ID of the transition
        """

        self.InState = InState
        self.InString = InString
        self.OutString = OutString
        self.OutState = OutState

    def Output(self):
        return [self.OutString, self.OutState]

In [62]:
class State:
    """ This is a class to define the properties of an individual State in a automata"""

    def __init__(self, name, FinalState):
        """ Initailize the State with Arguement:
            ID           -  Unique Identifier for the State (String or Integer)
            StateType    -  Defines the if the State is Terminal State or vice versa (Boolean)
            Transition   -  COntains all transitions from the State (Dictionary)"""

        self.ID = name
        self.StateType = FinalState
        self.Transition = dict()

    def addTransition(self, InString, OutString, OutState):
        """ Appends a Transition into the class variable 'Transition'
            Arguement of the method:
            Instring    -  Transistion string input 
            Outstring   -  Output string of the transition
            Outstate    -  Next state of the transition
            
            Return:
            None
            """

        if InString in self.Transition.keys():
            print("Transition with same Instring available")
            self.Transition[InString].append(Transition(self, InString,OutString= OutString, OutState = OutState))
        else:
            print("New input string Transition")
            self.Transition[InString] = []
            self.Transition[InString].append(Transition(self, InString, OutString = OutString, OutState = OutState))

    def CheckTransition(self, InputAlphabet):

        if InputAlphabet in self.Transition.keys() and '' in self.Transition.keys():
            Transition_Output_List = []
            for Transit in self.Transition[InputAlphabet]:
                Transition_Output_List.append(Transit.Output())
            for Transit in self.Transition['']:
                Transition_Output_List.append(Transit.Output())
            return Transition_Output_List

        elif InputAlphabet not in self.Transition.keys():
            print("No Transition as such")
            return [-1]
        else:

            Transition_Output_List = []
            for Transit in self.Transition[InputAlphabet]:
                Transition_Output_List.append(Transit.Output())
            return Transition_Output_List


In [53]:
class Automata:
    """ Basic class to define the automata's structure and its transitions. Further this automata model and be deployed in acceptor or automaton for applications."""

    def __init__(self, StateCount, Alphabet, OutAlphabets, InitialState, FinalStaeCount):
        
        self.StatesCount = StateCount
        self.Alphabet = Alphabet
        self.OutAlphabets = OutAlphabets
        self.InitialState = InitialState
        self.FinalStaeCount = FinalStaeCount

        self.States = dict()
    
    # Methods for building the automata structure

    def AddState(self, ID, Type):

        if self.StatesCount >= len(self.States):
            self.States[ID] = State(ID, Type)
        else:
            print("Exceeding the number of states!!!")

    def AddStateTransition(self, InState, InString, OutString, OutState):

        if InState in self.States.keys() and OutState in self.States.keys():
            In = self.States[InState]
            In.addTransition(InString, OutString, OutState)
            return [1]
        else:
            return [-1]

    # Create a new Transition object for a state

    def AddsetTransition(self, InState, set ,OutState):
        
        for a in set:
            PCode = self.AddStateTransition(InState, a, a, OutState)
            if PCode == -1:
                print(f'No state : {InState} present for the Transition |OR| No state : {OutState} present for the Transition')
                break
        if PCode != -1:
            print("Successfully appended transitions")
        
    def AddsetEpilsonTransition(self, InState, set ,OutState):

        for a in set:
            PCode = self.AddStateTransition(InState, a, '', OutState)
            if PCode == -1:
                print(f'No state : {InState} present for the Transition |OR| No state : {OutState} present for the Transition')
                break
            if PCode != -1:
                print("Successfully appended transitions")


    def AddEpilsonTransition(self, InState, OutState):

        PCode = self.AddStateTransition(InState, '', '', OutState)
        if PCode == -1:
            print(f'No state : {InState} present for the Transition |OR| No state : {OutState} present for the Transition')
        else:
            print("Successfully appended transitions")

    def AddEpilsonStringTransition(self, InState, OutString, OutState):

        PCode = self.AddStateTransition(InState, '', OutString, OutState)
        if PCode == -1:
            print(f'No state : {InState} present for the Transition |OR| No state : {OutState} present for the Transition')
        else:
            print("Successfully appended transitions")
        


In [103]:
AutoEx_1 = Automata(StateCount=3, Alphabet=set('01'), OutAlphabets= None, InitialState='a', FinalStaeCount=1)

# Adding States
AutoEx_1.AddState(ID='a', Type= False)
AutoEx_1.AddState(ID='b', Type= True)
AutoEx_1.AddState(ID='c', Type= True)

# Adding Transitions
AutoEx_1.AddStateTransition(InState='a', InString = '1', OutString = '0', OutState= 'b')
AutoEx_1.AddStateTransition(InState='a', InString = '1', OutString = '1', OutState= 'c')
AutoEx_1.AddStateTransition(InState='b', InString = '0', OutString = '1', OutState= 'c')

New input string Transition
Transition with same Instring available
New input string Transition


[1]

In [115]:
class Machine:

    def __init__(self, Automata, InputString):
        self.Automata = Automata
        self.InputString = InputString

    def Run(self,CurrentState, I = None, String = None):
        
        if I is None:
            if self.Automata.States[CurrentState].StateType == True:
                return [[True]]
            else:
                return [[False]]
        else:
            Output = []
            Transition_Outputs = self.Automata.States[CurrentState].CheckTransition(InputAlphabet = I)

            if Transition_Outputs[0] == -1:
                return [[False]]

            else:

                for Transit_Output in Transition_Outputs:
                    
                    if Transit_Output == -1:
                        continue

                    if len(String) == 0:
                        Sub_Transition_Output = self.Run(Transit_Output[1], I = None, String= None)
                    else:
                        Sub_Transition_Output = self.Run(Transit_Output[1], I = String[0], String= String[1:])
                    
                    for S in Sub_Transition_Output:
                        S.insert(0,Transit_Output[0])
                        Ouput = Output.append(S) 
                return Output


In [117]:
O = Machine(AutoEx_1,"00")
O.Run(AutoEx_1.InitialState, I=O.InputString[0],String=O.InputString[1:])

No Transition as such


[[False]]

In [ ]:
A = [1 ,2,4,2,4]
B = ["a", 1, True, False]

A+B
A = None
A == None

A = "asdfklsadlf"
A = ""
len(A)

0